## Introduction

Let's develop an evaluation methodology here and then finalize it as the script for the general use in the `src` folder. This approach is also likely to evolve a bit with the complexity of the future models, however, the main principles outlined here will stay the same.

One of the most important points in model evaluation here is selecting the data for the train and test datasets. Initial idea for this was to take the whole time range present in the data, and give the latest 20%-30% of it to be the test.

However, such an approach has a very serious drawback. As evident from the performed EDA, more than a half of users in our data do not spend long time on the platform and likely login just once. In fact, as fiscovered, almost half of the users have their earliest and latest rating events within an hour. This means that if we split just the general timestamps, than most of these short-period users will have no recommendations to make in test, and around 20-30% that end up in the "test period" fully will be essentially the new user predictions.

In fact, this is exactly the example how an entensive EDA can later help us to make quick decisions in the later development pipeline steps.

So, after reviewing a bit of literature of the topic (the papers [[1]](https://castells.github.io/papers/irj2020.pdf) and [[2]](https://arxiv.org/pdf/2007.13237) were particularly helpful, as they compared several approaches and hinted at the possible solution for our data) and looking at the data properties, the data split strategy was designed.

## Data split strategy

The motivation behind it is the following one. The temporal structure of our data is one of its most important properties and can contain a lot of information about the user. For the long-term users, sequence of the ratings will most likely means that the user has watched the movies in approximately that order, meaning that the decision to watch the later ones was based on the already watched films. Yes, it is also theoretically possible to recommend earlier movies based on the later ones, but this is less intuitive and natural way of recommending content. A good example would be a user that has watched a film in some new genre, didn't like it at all, and not as a result avoids this genre or this director at all. Therefore, a temporal dimension is important for our data, but if such a need arises it can also be omitted.

Interestingly, the short-term users rating sequence also can be indicative of the way they would like to watch them. The user is most likely to first rate the film that made the biggest impact to them, then rate the film that reminds them the previous one the most etc. However, we should be aware that in high-speed rating bursts the previous recommender system that possibly existed on the website from where the ratings are can affect the sequence by making the users rate the suggested movies. This can cause a but of the previous system leakage, but users still usually make a final movie selection themselves, thus "online validating" that system.

Therefore, the temporal dimension should definitely be present in out split. However, because we don't want the most users to be left out of test altogether and the other being totally new in the face of our system, we need to sample on the user-wise ratings, and not time-wise. For each user, we take their timespan of ratings, and assign the latest 20-30% (as defined by a parameter) to the test. This allows us to make informed test predictions about each user that has enough observations (at least 20 in our case), as is sometimes called a Temporal-User data split.

The biggest disadvantage of this method noted in several works is that is samples the same test time period for each user if we "normalize" their time on the platform, but their test periods still represent different absolute times. This can cause partial "future leakage" for the users other than this, even though it is much less than in the leave-one-out data split. The level of how biased the other user's "future" predictions are to this one is devatable, as it can be argued that they just give us more context on how the different users can operate.

Also, the danger in "future leakage" is in accidentally including the information that is somehow influenced by predictions. However, this is not the case for the other users here. Remember that we have no timestamps of the movies release, and, as such, we have no way of knowing when is the earliest time points when a certain movie can be watched. It does not correspond to the earliest timestamp the movie was rated. Therefore, we predict not "which of the future films this user can watch", but "which of the present in the data films can user watch." This is also evident from the fact, that our candidate films will be essentially sampled from the whole dataset. Because of this, the future ratings of *other* users do not really reveal anything about the test predictions of *this* user, as they just select from the whole movie list that we have available for the purpose of our dataset. There newer other user predictions just give more context how the sequence of ratings may go.

Worth noting that this would've been completely different if we would've had film release days in our data. In that case, the film list from which any user can select at any given point in time would be different, and would've had no choice but to hard cut the time after the test point's time.

Still, this no-release-time notion is artificial, and in the reality films can be realeased alongside our ratings. So even here a very small amount of "future leakage" from the other users can occur in the form of the new films, though again it can be argued that there points just provide new context about the users preferences. Most of the approaches from the reviewed literature [2] seem to not take unto account future leakage in the ratings systems at all, or for the rating of the other users.

### Evaluation approach algorithm

If we want to evaluate how our model would've worked at any point in time, we need to simulate the data as it was at that time. Therefore, for the model that allow such a recalculation, we will do the evaluation this way:

- input data: train data, test data, `UserID` to predict, `Timestamp` for which we make a prediction (test point time)
- cutoff the updated train data on the test point time (train data + test data that is "already known" at that time, example: we trained the model on the data up to 1st June, but the user have since watched 2 other movies that we've also made test on -> the recommendations need to count the previous observation, as with the time series), train the model just on that time
- make a prediction based on the capped train data
- form such predictions for all of the users
- calculate the dataset-wide metrics

The second point would be skipped in the future for the models that require lengthy training process (like deep learning approaches), allowing a small part of "future leakage" from the other users, as discussed earlier. For that reason, for the other models we will make two evaluations: for time cut-off train data and static train data.

The first one is the pure and most realistic model quality evaluation and will be used to compare all models where this approach will be possible. But the second one will be used in the future to compare the results with the models where model updating of the newly observed data for each point would be impossible.

## Metrics

The following metrics were selected to evaluate the recommendation quality. Each dataset-wide calculation can have several ways of aggregation. For example, metrics can be calculated for every user, or directly for all of the observations in the data. Our evaluation pipeline will have a parameter for this, but by default we do not average over the user's individual scores and calculate it directly. That changed during the development, and the main motivation for this is that the average score over users can "mask" a really bad score for some of them.

Also, several metrics highly depend on the number of points being evaluated. However, average-over-users metrics are good to confirm the overall performance. The usage really depends on the goal of the recommender system.

If we want to equally reward the system for satisfying each user (i.e. likely our profut depends on the number of users), the average is the way to go.

If we want to reward the best prediction regardless of the user, meaning that frequent users fill have much more weight (i.e. likely each "movie watch" gives us profit instead of a user subscription), than we will use the dataset-wide metric directly.

So, the metrics used for the model evaluation are:

- MAE:

In [1]:
from sklearn.metrics import mean_absolute_error

In [3]:
import numpy as np
import pandas as pd
import scipy

In [5]:
df_ratings = pd.read_csv('../../data/ml-1m/ratings.dat',
                         delimiter='::',
                         header=None,
                         names=['UserID','MovieID','Rating','Timestamp'],
                         engine ='python')

In [14]:
ratings_y = df_ratings[df_ratings['UserID'] == 1]['Rating'].tolist()
ratings_y_pred_example = np.random.uniform(1, 5, len(ratings_y))

In [15]:
mean_absolute_error(ratings_y, ratings_y_pred_example)

1.3464283466567633

- RMSE:

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [17]:
sqrt(mean_squared_error(ratings_y, ratings_y_pred_example))

1.7327511554413404

- precision (top-1, always calculated across the entire dataset)

In [21]:
def precision_special(y_true, y_pred):
    return np.sum(y_true == y_pred)/len(y_true)

In [22]:
precision_special(ratings_y, np.round(ratings_y_pred_example))

0.2641509433962264

- average precision:

In [23]:
def precision_top_k(y_true, y_pred, k):
    return np.sum(y_true[:k] == y_pred[:k])/k

In [24]:
def average_precision(y_true, y_pred, m):
    return np.sum([precision_top_k(y_true, y_pred, k) for k in range(1,m+1)])/m

In [82]:
def argsort_top_n(y_list, n):
    indices_unsorted = np.argpartition(y_list, -n)[-n:]
    combined_time_order_unsorted = np.array([np.array(y_list)[indices_unsorted], -indices_unsorted]).T
    # print(f'--{combined_time_order_unsorted} {combined_time_order_unsorted[:,::-1].T}')
    # print(f'--{np.lexsort(combined_time_order_unsorted[:,::-1].T)}')
    # print(f'--{np.lexsort(combined_time_order_unsorted[:,::-1].T)}')
    return indices_unsorted[np.lexsort(combined_time_order_unsorted[:,::-1].T)][::-1]

In [83]:
argsort_top_n(ratings_y, 20)

array([ 0,  4,  6,  7, 10, 14, 18, 22, 23, 25, 36, 37, 39, 40, 41, 45, 46,
       48, 17, 19])

In [84]:
np.array(ratings_y)[argsort_top_n(ratings_y, 20)]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4])

As we can see, ranking metrics can have a bit of a problem in our data, as ground truth ratings have really small discretization and the ratings of 5 are all really the top-rated ground truth recommendations. Therefore, it only really makes sense to use the average precision and other order- or rank-based approaches for evaluating the results of the top-N predictions, where N is the number of test predictions for the user. Alternatively, we can calculate (unoredered) precision for each user for the top-M predictions, where M is the number of highest-ranked ground truth predictions in the test set for this user.

We can theoretically say that the movie the user ranked with 5/5 first would be a better prediction than the next 5/5-ranked movies (as implemented in the `argsort_top_n` function above used to convert the ratings into ranked item predictions), but that is not exactly true. This brings all of the ranked metrics in a bit of a question here.

In [41]:
average_precision(argsort_top_n(ratings_y, 20),
                  argsort_top_n(np.round(ratings_y_pred_example), 20),
                  20)

0.06572031619051742

- mean reciprocal rank (calculated for each test point based on which position the correct item appears among the ranked predictions):

In [187]:
def mean_reciprocal_rank(relevant_items_list, predicted_candidates_lists):
    ranks = [list(predicted_candidates_lists[i_i]).index(
        item)+1 if item in predicted_candidates_lists[i_i] else 0 for i_i, item in enumerate(relevant_items_list)]
    return np.sum([1/rank if rank > 0 else 0 for rank in ranks])/len(relevant_items_list)

In [50]:
mean_reciprocal_rank([0, 1, 2], [[1, 2], [1, 2], [1, 2]]) # (0 + 1 + 0.5)/3 = 0.5

0.5

- NDCG (calculated for the top-M predictions for the user):

In [86]:
from sklearn.metrics import ndcg_score

In [107]:
ndcg_score(np.array(ratings_y)[argsort_top_n(ratings_y, 20)].reshape(1,-1),
           np.array(ratings_y_pred_example)[argsort_top_n(ratings_y_pred_example, 20)].reshape(1,-1))

1.0

This demonstrates the issue with the ranked metrics for this specific dataset in the case we have many values with the rating 5: the metrics simply struggle to identify the correct ranking of the ground truth ratings.

- Coverage (of the all elements in the train dataset). Is calculated as a simple proportion.

Also, as we only have on offline evaluation, our hit rate metric is essentially the same as precision.

Now, let's develop a whole evaluation pipeline:

In [140]:
df_ratings.groupby('UserID', group_keys=False).apply(lambda x: x.tail(int(np.round(x.shape[0]*0.2))))

/tmp/ipykernel_466877/3060180110.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ratings.groupby('UserID', group_keys=False).apply(lambda x: x.tail(int(np.round(x.shape[0]*0.2))))


,UserID,MovieID,Rating,Timestamp
42,1,1962,4,978301753
43,1,2692,4,978301570
44,1,260,4,978300760
45,1,1028,5,978301777
46,1,1029,5,978302205
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [137]:
df_ratings.groupby('UserID').count().head()

,MovieID,Rating,Timestamp
UserID,,,
1,53,53,53
2,129,129,129
3,51,51,51
4,21,21,21
5,198,198,198


In [138]:
df_ratings.groupby('UserID', group_keys=False).apply(lambda x: x.tail(int(np.round(x.shape[0]*0.2)))).groupby('UserID').count().head()

/tmp/ipykernel_466877/3546730996.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ratings.groupby('UserID', group_keys=False).apply(lambda x: x.tail(int(np.round(x.shape[0]*0.2)))).groupby('UserID').count().head()


,MovieID,Rating,Timestamp
UserID,,,
1,11,11,11
2,26,26,26
3,10,10,10
4,4,4,4
5,40,40,40


In [144]:
df_ratings.loc[42]

UserID               1
MovieID           1962
Rating               4
Timestamp    978301753
Name: 42, dtype: int64

The pipeline below was a result of extensive development and experimentation and took almost 70% of the time spent on taskts 1 and 2 of the first submission. Here is just the final version which is also doubled in the `src/evaluation.py`, most of the development process took place in parallel to the mean-rating baseline model development as they allow to test each other, so it can be reviewed in the `Exp_1_4` folder under the name `experiment_dev_evaluation.py`.

In [4]:
class EvaluationPipeline:
    def __init__(self,
                 total_rating_data,
                 train_test_split: float = 0.2):
        self.total_rating_data = total_rating_data
        self.train_test_split = train_test_split
        self.test_data = self.get_test_data(self.total_rating_data)
        self.train_data = self.total_rating_data[~self.total_rating_data.index.isin(self.test_data.index)]

    def get_test_data(self, total_data):
        return total_data.groupby('UserID', group_keys=False).apply(
            lambda x: x.tail(int(np.round(x.shape[0]*self.train_test_split))))

    def evaluate(self,
                 model_object,
                 metrics_list = False,
                 user_average_metrics: bool = False,
                 retrain_model_each_point: bool = False):
        if not metrics_list:
            metrics_list = ['mae','rmse','precision','average_precision',
                            'mean_reciprocal_rank','ndcg','coverage']
        if user_average_metrics:
            recommendation_results = {}
            ratings_y_pred = {}
        else:
            recommendation_results = []
            ratings_y_pred = []
        sorted_total_data_test = self.total_rating_data.sort_values('Timestamp')
        rows_before_timestamp_test = sorted_total_data_test.groupby('Timestamp').count()[
            'UserID'].cumsum().shift(1).fillna(0).astype(int).to_dict()
        if retrain_model_each_point:
            sorted_data_train = self.train_data.sort_values('Timestamp')
            rows_before_timestamp_train = sorted_data_train.groupby('Timestamp').count()[
                'UserID'].cumsum().shift(1).fillna(0).astype(int)
            timestamps_not_in_train = [timestamp for timestamp in rows_before_timestamp_test.keys(
                ) if timestamp not in rows_before_timestamp_train.index]
            rows_before_timestamp_train = pd.concat([rows_before_timestamp_train,
                                                     pd.Series(None,
                                                               index=timestamps_not_in_train)])
            rows_before_timestamp_train = rows_before_timestamp_train.sort_index().ffill().astype(int)
        for i_p, test_point in tqdm(self.test_data.iterrows(),
                                    total=self.test_data.shape[0]):
            test_point_timestamp = test_point['Timestamp']
            test_point_user = test_point['UserID']
            if user_average_metrics and test_point_user not in recommendation_results.keys():
                recommendation_results[test_point_user] = []
                ratings_y_pred[test_point_user] = []
            if retrain_model_each_point:
                train_data_each_point = sorted_data_train[:rows_before_timestamp_train[test_point_timestamp]]
                model_object.fit(train_data_each_point)
            # Using the whole dataset, as for each new test point for the user previous test points matter too
            items_pred, ratings_pred = model_object.predict(sorted_total_data_test[
                                                                :rows_before_timestamp_test[test_point_timestamp]],
                                                            test_point_user,
                                                            test_point_timestamp)
            if user_average_metrics:
                recommendation_results[test_point_user].append([i_p, items_pred])
                ratings_y_pred[test_point_user].append(ratings_pred[items_pred.tolist().index(
                    test_point_user)] if (test_point_user in items_pred) else 0)
            else:
                recommendation_results.append([i_p, items_pred])
                ratings_y_pred.append(ratings_pred[items_pred.tolist().index(
                    test_point_user)] if (test_point_user in items_pred) else 0)
        metrics_output_dict = {}
        if not user_average_metrics:
            ratings_y_true = [self.test_data.loc[
                              test_point_index, 'Rating'] for test_point_index in self.test_data.index]
            # ratings_y_pred = [ratings_pred[items_pred.index(
            #     self.test_data.loc[test_point_index, 'movieID'])] if self.test_data.loc[
            #         test_point_index, 'movieID'] in items_pred else 0 for test_point_index in self.test_data.index]
            # ratings_y_pred = [recommendation_results[i_p][2][recommendation_results[i_p][1].tolist().index(
            #     self.test_data.loc[test_point_index, 'MovieID'])] if self.test_data.loc[
            #         test_point_index, 'MovieID'] in recommendation_results[i_p][
            #             1] else 0 for i_p, test_point_index in enumerate(self.test_data.index)]
            self.test_data['Rating pred'] = ratings_y_pred
            ratings_y_true_users = self.test_data.groupby('UserID')['Rating'].apply(list).to_dict()
            ratings_y_pred_users = self.test_data.groupby('UserID')['Rating pred'].apply(list).to_dict()
            largest_user_id_total = self.total_rating_data['UserID'].max()
            items_id_pred = [pred[1][0] if len(pred[1]) > 0 else (
                largest_user_id_total + 1) for pred in recommendation_results]
            for metric in metrics_list:
                if metric == 'mae':
                    metrics_output_dict['mae'] = mean_absolute_error(ratings_y_true, ratings_y_pred)
                elif metric == 'rmse':
                    metrics_output_dict['rmse'] = sqrt(mean_squared_error(ratings_y_true, ratings_y_pred))
                elif metric == 'precision':
                    metrics_output_dict['precision'] = precision_special(self.test_data['MovieID'].to_numpy(),
                                                                         items_id_pred)
                elif metric == 'average_precision':
                    average_precision_list = []
                    for user in self.test_data['UserID'].unique():
                        m_user = len(ratings_y_true_users[user])
                        average_precision_list.append(average_precision(
                            argsort_top_n(ratings_y_true_users[user], m_user),
                            argsort_top_n(ratings_y_pred_users[user], m_user),
                            m_user))
                    metrics_output_dict['average_precision'] = np.mean(average_precision_list)
                elif metric == 'mean_reciprocal_rank':
                    metrics_output_dict['mean_reciprocal_rank'] = mean_reciprocal_rank(
                        self.test_data['MovieID'].to_numpy(),
                        [pred[1] for pred in recommendation_results])
                elif metric == 'ndcg':
                    ndcg = []
                    for user in self.test_data['UserID'].unique():
                        m_user = len(ratings_y_true_users[user])
                        if m_user > 1:
                            # NDCG only is defined is there is more than 1 point
                            ndcg.append(ndcg_score(
                                np.array(ratings_y_true_users[user])[
                                    argsort_top_n(ratings_y_true_users[user], m_user)].reshape(1,-1),
                                np.array(ratings_y_pred_users[user])[
                                    argsort_top_n(ratings_y_pred_users[user], m_user)].reshape(1,-1)))
                    if len(ndcg) > 0:
                        metrics_output_dict['ndcg'] = np.mean(ndcg)
                    else:
                        metrics_output_dict['ndcg'] = 0.0
                elif metric == 'coverage':
                    items_train_unique = self.train_data['UserID'].unique()
                    metrics_output_dict['coverage'] = len(np.unique([
                        item for item in items_id_pred if item in items_train_unique]))/len(items_train_unique)
        return metrics_output_dict # recommendation_results

In [165]:
import sys
import os.path as osp

PROJECT_DIR = '../../'
PROJECT_DIR = osp.abspath(PROJECT_DIR)
print(PROJECT_DIR in sys.path)
if PROJECT_DIR not in sys.path:
    print(f'Adding project directory to the sys.path: {PROJECT_DIR!r}')
    sys.path.insert(1, PROJECT_DIR)

True


In [168]:
from src.evaluation import EvaluationPipeline

In [170]:
eval_baseline = EvaluationPipeline(df_ratings, 0.2)

In [171]:
eval_baseline